# 📝 Exercise M6.01

The aim of this notebook is to investigate if we can tune the hyperparameters
of a bagging regressor and evaluate the gain obtained.

We will load the California housing dataset and split it into a training and
a testing set.

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

data, target = fetch_california_housing(as_frame=True, return_X_y=True)
target *= 100  # rescale the target in k$
data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=0, test_size=0.5)

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">If you want a deeper overview regarding this dataset, you can refer to the
Appendix - Datasets description section at the end of this MOOC.</p>
</div>

Create a `BaggingRegressor` and provide a `DecisionTreeRegressor`
to its parameter `base_estimator`. Train the regressor and evaluate its
statistical performance on the testing set using the mean absolute error.

In [8]:
# Write your code here.
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import cross_validate

bagged_trees = BaggingRegressor(
    base_estimator=DecisionTreeRegressor()
)

cv_result = cross_validate(bagged_trees, data_train, target_train,
                           cv=10, scoring='neg_mean_absolute_error')

print(f"{-cv_result['test_score'].mean():.3f} +/- {cv_result['test_score'].std():.3f}")

36.426 +/- 1.267


Now, create a `RandomizedSearchCV` instance using the previous model and
tune the important parameters of the bagging regressor. Find the best
parameters  and check if you are able to find a set of parameters that
improve the default regressor still using the mean absolute error as a
metric.

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">You can list the bagging regressor's parameters using the <tt class="docutils literal">get_params</tt>
method.</p>
</div>

In [10]:
# Write your code here.
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    "n_estimators": randint(10, 30),
    "max_samples": [0.5, 0.8, 1.0],
    "max_features": [0.5, 0.8, 1.0],
    "base_estimator__max_depth": randint(3, 10),
}
search = RandomizedSearchCV(
    bagged_trees, param_grid, n_iter=20, scoring="neg_mean_absolute_error"
)
_ = search.fit(data_train, target_train)

In [11]:
import pandas as pd

columns = [f"param_{name}" for name in param_grid.keys()]
columns += ["mean_test_score", "std_test_score", "rank_test_score"]
cv_results = pd.DataFrame(search.cv_results_)
cv_results = cv_results[columns].sort_values(by="rank_test_score")
cv_results["mean_test_score"] = -cv_results["mean_test_score"]
cv_results

,param_n_estimators,param_max_samples,param_max_features,param_base_estimator__max_depth,mean_test_score,std_test_score,rank_test_score
8,17,0.8,0.8,9,39.358535,1.398588,1
18,18,0.5,1.0,9,39.931254,0.959234,2
3,24,1.0,1.0,8,40.686070,0.975481,3
1,24,1.0,1.0,8,40.765162,1.348873,4
5,17,0.8,0.8,8,41.175155,1.067822,5
15,26,1.0,1.0,7,42.773520,1.174494,6
14,16,1.0,0.8,7,43.192972,1.415019,7
19,11,1.0,0.8,7,43.463934,0.947514,8
9,25,0.5,1.0,6,45.066402,1.018933,9
13,15,0.8,0.5,9,45.692113,2.134594,10


We see that the bagging regressor provides a predictor in which fine tuning
is not as important as in the case of fitting a single decision tree.